## Branch Getters

In [1]:
import os 
os.chdir('..')

In [2]:
from utils.format.schema import check_argument, convert_to_bytes_based_on_schema, convert_from_bytes_based_on_schema
import inspection.branch as inspection_branch

### GETTERS WITHOUT RPC SERVER CALLS

#### Get Branch Name From Branch Id

In [3]:
branch_id = 'AVESAoKD' ## replace with actual id 
kwargs = dict(branch_id=branch_id)
arguments_valid_schema = check_argument(arg=kwargs, schema=inspection_branch.get_branch_name_from_branch_id_schema)
print("arguments are valid according to schema: ", arguments_valid_schema)
converted_kwargs = convert_to_bytes_based_on_schema(schema=inspection_branch.get_branch_name_from_branch_id_schema, data=kwargs)
print("converted_kwargs: ", converted_kwargs)
response = inspection_branch.get_branch_name_from_branch_id(**converted_kwargs)
decoded_response = convert_from_bytes_based_on_schema(schema=inspection_branch.get_branch_name_from_branch_id_schema["response"], data=response)
print("decoded response: ", decoded_response)

arguments are valid according to schema:  True
converted_kwargs:  {'branch_id': b'\x01Q\x12\x02\x82\x83'}
decoded response:  Genesis Branch


In [4]:
# branch_id_bytes = b'\x01Q\x12\x02\x82\x83'
kwargs = dict(branch_id=branch_id, deserialize_buckets=False)
arguments_valid_schema = check_argument(arg=kwargs, schema=inspection_branch.get_branch_data_from_branch_id_schema)
print("arguments are valid according to schema: ", arguments_valid_schema)
converted_kwargs = convert_to_bytes_based_on_schema(schema=inspection_branch.get_branch_data_from_branch_id_schema, data=kwargs)
print("converted_kwargs: ", converted_kwargs)
response = inspection_branch.get_branch_data_from_branch_id(**converted_kwargs)
decoded_response = convert_from_bytes_based_on_schema(schema=inspection_branch.get_branch_data_from_branch_id_schema["response"], data=response)
print("decoded response: ", decoded_response)
# inspection_branch.get_config_from_branch_id(branch_id_bytes)


arguments are valid according to schema:  True
converted_kwargs:  {'branch_id': b'\x01Q\x12\x02\x82\x83', 'deserialize_buckets': False}
decoded response:  {'id': 'AVESAoKD', 'ns': '+2IAAA==', 'name': 'Genesis Branch', 'parent_id': '', 'stable_head': {'parent_submit_id': 'AVESArpTDQT7YgAA', 'submit_msg': 'Initial Submit'}, 'config': {'accept_conflicts': False, 'branch_type': 1}, 'sprouts': [], 'sprout_selection': [], 'name_resolution': 'AVESAk7VEAT7YgAA', 'interaction': 'AVESAjSNEQT7YgAA', 'signature': '', 'creation_ts': 1705160930, 'submit_trace': {'new_buckets': ['AVESAkVJAQ==', 'AVESAhOQAQ==', 'AVESAv3QAg=='], 'new_registered_names': [{'name': 'Dummy Article Name', 'id': 'AVESAv3QAg=='}]}}


### Getters via RPC server call

In [8]:
import requests
import json
from config.rpc_cfg import RPC_PORT

In [9]:
def json_rpc_call(method, params=None):
    url = f"http://localhost:{RPC_PORT}/"
    headers = {'content-type': 'application/json'}
    payload = {
        "method": method,
        "params": params,
        "jsonrpc": "2.0",
        "id": 0,
    }
    encoded_payload = json.dumps(payload).encode('utf-8')
    response = requests.post(url, data=encoded_payload, headers=headers)
    # print(response)#
    return response.json()

In [10]:
branch_id = 'AVESAvtC'
kwargs = dict(branch_id=branch_id)

In [11]:
kwargs_rpc = list(kwargs.values())
response = json_rpc_call(method="get_branch_name_from_branch_id", params=kwargs_rpc)
# response is inside the result key
name = response['result']
print("Test 'get_branch_name_from_branch_id' passed with branch name:", name)

Test 'get_branch_name_from_branch_id' passed with branch name: Genesis Branch
